Listing of All Businesses
https://data.lacity.org/Administration-Finance/Listing-of-All-Businesses/r4uk-afju/about_data


In [1]:
import pandas as pd

In [2]:
file_path = 'dataset\\Listing_of_All_Businesses_20250202.csv'
df = pd.read_csv(file_path)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1593026 entries, 0 to 1593025
Data columns (total 16 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   LOCATION ACCOUNT #         1593026 non-null  object 
 1   BUSINESS NAME              1593026 non-null  object 
 2   DBA NAME                   245872 non-null   object 
 3   STREET ADDRESS             1592924 non-null  object 
 4   CITY                       1592920 non-null  object 
 5   ZIP CODE                   1593026 non-null  object 
 6   LOCATION DESCRIPTION       1592903 non-null  object 
 7   MAILING ADDRESS            359466 non-null   object 
 8   MAILING CITY               359478 non-null   object 
 9   MAILING ZIP CODE           359344 non-null   object 
 10  NAICS                      624448 non-null   float64
 11  PRIMARY NAICS DESCRIPTION  624448 non-null   object 
 12  COUNCIL DISTRICT           1593026 non-null  int64  
 13  LOCATION STA

In [3]:
df

,LOCATION ACCOUNT #,BUSINESS NAME,DBA NAME,STREET ADDRESS,CITY,ZIP CODE,LOCATION DESCRIPTION,MAILING ADDRESS,MAILING CITY,MAILING ZIP CODE,NAICS,PRIMARY NAICS DESCRIPTION,COUNCIL DISTRICT,LOCATION START DATE,LOCATION END DATE,LOCATION
0,0003217523-0001-2,MARIA OFELIA NUNEZ,LA CAMPANA HARDWARE,1017 W 88TH STREET,LOS ANGELES,90044-2119,1017 88TH 90044,1017 W 88TH STREET,LOS ANGELES,90044-2119,444130.0,Hardware stores,0,10/01/2020,NaN,"(33.9581, -118.2926)"
1,0003211603-0001-0,ONE STOP HOLISTIC HEALTH LLC,NaN,1506 W MANCHESTER AVENUE,LOS ANGELES,90047-5424,1506 MANCHESTER 90047,1506 W MANCHESTER AVENUE,LOS ANGELES,90047-5424,NaN,NaN,8,09/03/2020,NaN,"(33.9597, -118.3028)"
2,0003109317-0001-3,GINA BRADFORD,NaN,7847 CANBY AVENUE,RESEDA,91335-2096,7847 CANBY 91335,7847 CANBY AVENUE,RESEDA,91335-2096,NaN,NaN,4,01/01/2018,NaN,"(34.2133, -118.535)"
3,0003241915-0001-6,SKAB INC,NaN,21731 VENTURA BLVD SUITE #300,WOODLAND HILLS,91364-1851,21731 Ventura 91364-1851,NaN,NaN,NaN,532290.0,Other consumer goods rental,3,01/01/2020,NaN,"(34.1692, -118.6016)"
4,0002982075-0001-1,PRODYNAMICS INC,TITAN PHYSICAL THERAPY AND WELLNESS,12099 W WASHINGTON UNIT #408,LOS ANGELES,90066-0000,12099 WASHINGTON 90066-0000,NaN,NaN,NaN,621340.0,"Offices of physical, occupational & speech the...",11,01/01/2017,NaN,"(33.9982, -118.4238)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1593021,0003257195-0001-0,MARIA MARTINEZ,ZARAGOZA JANITORIAL SERVICES,9555 LOCUST AVENUE,FONTANA,92335-6144,9555 LOCUST 92335,9555 LOCUST AVENUE,FONTANA,92335-6144,NaN,NaN,0,06/10/2021,NaN,"(34.0803, -117.4092)"
1593022,0002569692-0001-0,MAY NATIONAL INC,NaN,1425 1ST STREET,SAN FERNANDO,91340-2705,1425 1ST 91340-2705,NaN,NaN,NaN,NaN,NaN,0,02/19/2009,03/01/2015,NaN
1593023,0003069957-0001-0,LUCY RAMOS,ZUMBA MOMMA ZUMBA,3218 WABASH AVENUE,LOS ANGELES,90063-2722,3218 WABASH 90063,NaN,NaN,NaN,NaN,NaN,0,08/15/2018,NaN,"(34.0476, -118.1923)"
1593024,0003400956-0001-3,TROJAN SMOKE SHOP,NaN,2508 S FIGUEROA STREET,LOS ANGELES,90007-,2508 FIGUEROA 90007,2508 S FIGUEROA STREET,LOS ANGELES,90007,NaN,NaN,0,11/01/2023,NaN,NaN


### NAICS information
data collection from https://www.naics.com/search/#naics

In [3]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd

#### Attempt 1 
Get 2 digits info from webpage

In [4]:
web_url = 'https://www.naics.com/search/#naics'
html_content = requests.get(web_url).text
soup = BS(html_content, 'html.parser')

In [5]:
data = []
for code in soup.find_all('td', attrs = {'class': "first_child noWrap"}):
    sector_title = code.find_next('td')
    num_us_entities = code.find_next('td').find_next('td')
    #print(code.string, '\t', sector_title.string, '\t\t', num_us_entities.string)
    data.append([code.string.strip() if code.string else '',
                 sector_title.string.strip() if sector_title.string else '',
                 num_us_entities.string.strip() if num_us_entities.string else ''
                 ])    

In [6]:
df = pd.DataFrame(data, columns=['Code', 'Sector_Title', 'Number_of_US_Entities'])
df.to_csv('naics_dict.csv', index=False) 

#### Attempt 2
Get links from webpage for more detailed NAICS information

Get detailed NAICS information from sub-links then combine to bigger dataset

In [15]:
# function get NAICS info from detailed url
def naics_data_collect(web_url):
    html_content = requests.get(web_url).text
    soup = BS(html_content, 'html.parser')
    data = []
    for code in soup.find_all('div', attrs = {'class': "naicscode"}):
        sector_title = code.find_next('div')
        num_us_entities = code.find_next('div').find_next('div')
        data.append([code.string.strip() if code.string else '',
                    sector_title.string.strip() if sector_title.string else '',
                    num_us_entities.string.strip() if num_us_entities.string else ''
                    ])
    df = pd.DataFrame(data, columns=['Code', 'Sector_Title', 'Number_of_US_Entities'])
    return df

In [8]:
# attempt to save all naics links to set for later use
web_url = 'https://www.naics.com/search/#naics'
html_content = requests.get(web_url).text
soup = BS(html_content, 'html.parser')
links = set()
for content in soup.find_all('td', attrs = {'class': "first_child noWrap"}):
    a_tag = content.find('a')
    if a_tag:
        link = a_tag.get('href')
        links.add(link)
# links contain all sublink of 6 digits naics page

In [25]:
df_list = []
for link in links:
    temp = naics_data_collect(link)
    df_list.append(temp)

df_all = pd.concat(df_list, ignore_index=True)

*Note:* find out why duplicates records appear?

In [29]:
df_all.drop_duplicates(inplace=True)
df_all = df_all.iloc[1:,:]

In [30]:
# save all naics info to csv
df_all.to_csv('naics_all_dict.csv', index=False)

In [31]:
# break down total dataset to 6 digits, 4 digits, ...
df_6 = df_all[df_all['Code'].str.len() == 6]
df_4 = df_all[df_all['Code'].str.len() == 4]
df_2 = df_all[(df_all['Code'].str.len() != 6) & (df_all['Code'].str.len() != 4)]
df_6.to_csv('naics_6.csv', index=False)
df_4.to_csv('naics_4.csv', index=False)
df_2.to_csv('naics_2.csv', index=False)

### NAICS information clean up

In [2]:
import pandas as pd

In [3]:
file_path = 'dataset\\naics_dict.csv'

In [11]:
naics_2 = pd.read_csv(file_path)

In [12]:
naics_2

,Code,Sector_Title,Number_of_US_Entities
0,11,"Agriculture, Forestry, Fishing and Hunting","367,959"
1,21,Mining,"33,339"
2,22,Utilities,"52,270"
3,23,Construction,"1,512,763"
4,31-33,Manufacturing,"660,640"
5,42,Wholesale Trade,"698,477"
6,44-45,Retail Trade,"1,870,617"
7,48-49,Transportation and Warehousing,"711,582"
8,51,Information,"370,463"
9,52,Finance and Insurance,"771,419"


In [7]:
naics_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Code                   20 non-null     object
 1   Sector_Title           20 non-null     object
 2   Number_of_US_Entities  20 non-null     object
dtypes: object(3)
memory usage: 612.0+ bytes


In [13]:
# function deal with naics code is range.
def code_range(code):
    if '-' in code:
        start, end = code.split('-')
        # code is range not single number, get every number between them.
        return [str(i) for i in range(int(start), int(end) + 1)]
    else:
        return [code]

In [14]:
naics_2['Code'] = naics_2['Code'].apply(code_range)

# DataFrame.explode(column, ignore_index=False)
# Transform each element of a list-like to a row, replicating index values.
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.explode.html#pandas.DataFrame.explode
expanded_df = naics_2.explode('Code').reset_index(drop=True)
expanded_df.to_csv('naics_2_clean.csv', index=False)